In [14]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F
from awsglue.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 58613647-7693-480c-9931-028cb7b6fa42.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 58613647-7693-480c-9931-028cb7b6fa42.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 58613647-7693-480c-9931-028cb7b6fa42.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 58613647-7693-480c-9931-028cb7b6fa42.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [15]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='database_name', table_name='table_name')
dyf.printSchema()

+--------------------+--------------------+--------------------+
|        Product Name|               Brand|             Reviews|
+--------------------+--------------------+--------------------+
|Eilish Eau de Parfum|       Billie Eilish|4.5 out of 5 star...|
|COCO MADEMOISELLE...|              CHANEL|4.5 out of 5 star...|
|Donna Born In Rom...|           Valentino|4.8 out of 5 star...|
| Cloud Eau de Parfum|       Ariana Grande|4.3 out of 5 star...|
|Miss Dior Eau de ...|                Dior|4.6 out of 5 star...|
|Born in Roma Donn...|           Valentino|4.8 out of 5 star...|
|   Her Eau de Parfum|            Burberry|4.5 out of 5 star...|
|CHANCE EAU TENDRE...|              CHANEL|4.4 out of 5 star...|
|Burberry Goddess ...|            Burberry|4.7 out of 5 star...|
|Good Girl Blush E...|    Carolina Herrera|4.5 out of 5 star...|
|MOD Vanilla Eau d...|       Ariana Grande|4.3 out of 5 star...|
|Eilish No. 2 Eau ...|       Billie Eilish|4 out of 5 stars ...|
|CHANCE EAU TENDRE...|   

In [16]:
# Use regexp_extract to extract the rating and number of reviews
df = df.withColumn("Rating", regexp_extract(col("Reviews"), r'(\d+\.\d) out of 5 stars', 1)) \
       .withColumn("Number of Reviews", regexp_extract(col("Reviews"), r'; (\d+) reviews', 1).cast("int"))

# Drop the original 'Reviews' column
df = df.drop("Reviews")

# Show the DataFrame
df.show(truncate=False)

+---------------------------------------+-----------------------+------+-----------------+
|Product Name                           |Brand                  |Rating|Number of Reviews|
+---------------------------------------+-----------------------+------+-----------------+
|Eilish Eau de Parfum                   |Billie Eilish          |4.5   |3431             |
|COCO MADEMOISELLE Eau de Parfum Spray  |CHANEL                 |4.5   |362              |
|Donna Born In Roma Eau de Parfum       |Valentino              |4.8   |6034             |
|Cloud Eau de Parfum                    |Ariana Grande          |4.3   |2772             |
|Miss Dior Eau de Parfum                |Dior                   |4.6   |9704             |
|Born in Roma Donna Perfume Gift Set    |Valentino              |4.8   |6034             |
|Her Eau de Parfum                      |Burberry               |4.5   |5046             |
|CHANCE EAU TENDRE Eau de Parfum Spray  |CHANEL                 |4.4   |268              |

In [21]:
# Calculate the weighted rating for each product
df_with_weighted_rating = df.withColumn(
    "Weighted Rating", 
    F.col("Rating") * F.col("Number of Reviews")
)

# Show the DataFrame with the new "Weighted Rating" column
df_with_weighted_rating.show(truncate=False)

+---------------------------------------+-----------------------+------+-----------------+------------------+
|Product Name                           |Brand                  |Rating|Number of Reviews|Weighted Rating   |
+---------------------------------------+-----------------------+------+-----------------+------------------+
|Eilish Eau de Parfum                   |Billie Eilish          |4.5   |3431             |15439.5           |
|COCO MADEMOISELLE Eau de Parfum Spray  |CHANEL                 |4.5   |362              |1629.0            |
|Donna Born In Roma Eau de Parfum       |Valentino              |4.8   |6034             |28963.2           |
|Cloud Eau de Parfum                    |Ariana Grande          |4.3   |2772             |11919.6           |
|Miss Dior Eau de Parfum                |Dior                   |4.6   |9704             |44638.399999999994|
|Born in Roma Donna Perfume Gift Set    |Valentino              |4.8   |6034             |28963.2           |
|Her Eau d

In [29]:
# Filter out rows where Rating is NULL or empty
df_clean = df.filter(df['Rating'].isNotNull() & (df['Rating'] != ''))

# Sort by Rating and number of Reviews in descending/ascending order and show the top 10
top_10_by_rating = df_clean.orderBy(F.col("Rating"), ascending=False).limit(10)
bottom_10_by_rating = df_clean.orderBy(F.col("Rating"), ascending=True).limit(10)
top_10_by_reviews = df_clean.orderBy(F.col("Number of Reviews"), ascending=False).limit(10)
bottom_10_by_reviews = df_clean.orderBy(F.col("Number of Reviews"), ascending=True).limit(10)

# Show the results
print("Top 10 Perfumes Based on Rating (Ignoring Empty Ratings):")
top_10_by_rating.show(truncate=False)

print("Bottom 10 Perfumes Based on Rating (Ignoring Empty Ratings):")
bottom_10_by_rating.show(truncate=False)

print("Top 10 Perfumes Based on Number of Reviews (Ignoring Empty Ratings):")
top_10_by_reviews.show(truncate=False)

print("Bottom 10 Perfumes Based on Number of Reviews (Ignoring Empty Ratings):")
bottom_10_by_reviews.show(truncate=False)


Top 10 Perfumes Based on Rating (Ignoring Empty Ratings):
+-------------------------------------------+------------------+------+-----------------+
|Product Name                               |Brand             |Rating|Number of Reviews|
+-------------------------------------------+------------------+------+-----------------+
|Wild Poppy Eau de Parfum                   |NEST New York     |4.9   |68               |
|J'adore Eau Lumière Eau de Toilette        |Dior              |4.8   |100              |
|Pleasures Eau de Parfum Perfume Spray      |Estée Lauder      |4.8   |699              |
|Mini Flowerbomb Perfume Set                |Viktor&Rolf       |4.8   |128              |
|Donna Born In Roma Eau de Parfum           |Valentino         |4.8   |6034             |
|Libre Flower & Flames Eau de Parfum Florale|Yves Saint Laurent|4.8   |1762             |
|Libre Eau de Parfum Intense                |Yves Saint Laurent|4.8   |1018             |
|Bright Crystal Absolu Eau de Parfum      

In [30]:
# Calculate the weighted rating (rating * number of reviews) for each row
df_with_weights = df.withColumn(
    "Weighted Rating", 
    F.col("Rating") * F.col("Number of Reviews")
)

# Group by Brand and calculate the total number of reviews and the weighted sum of ratings
brand_avg_reviews_df = df_with_weights.groupBy("Brand").agg(
    F.avg("Rating").alias("Average Rating"),  # Average rating for each brand
    F.sum("Number of Reviews").alias("Total Number of Reviews")  # Total reviews for each brand
)

# Sort results
top_10_df = brand_avg_reviews_df.orderBy(F.col("Average Rating").desc()).limit(10)
bottom_10_df = brand_avg_reviews_df.orderBy(F.col("Average Rating").asc()).limit(10)

print("Top 10 Brands by Average Rating:")
top_10_df.show(truncate=False)

print("Bottom 10 Brands by Average Rating:")
bottom_10_df.show(truncate=False)


Top 10 Brands by Average Rating:
+-----------------+-----------------+-----------------------+
|Brand            |Average Rating   |Total Number of Reviews|
+-----------------+-----------------+-----------------------+
|florence by mills|4.8              |166                    |
|Guerlain         |4.8              |138                    |
|Valentino        |4.733333333333333|17298                  |
|Sol de Janeiro   |4.7              |5106                   |
|DIME             |4.7              |15967                  |
|RMS Beauty       |4.7              |123                    |
|Donna Karan      |4.699999999999999|2137                   |
|Prada            |4.675            |17220                  |
|Snif             |4.673333333333333|6217                   |
|Viktor&Rolf      |4.655555555555555|24594                  |
+-----------------+-----------------+-----------------------+

Bottom 10 Brands by Average Rating:
+-----------------+-----------------+-----------------------+


In [35]:
# Convert Spark DataFrame to Glue DynamicFrame
dynamic_frame = DynamicFrame.fromDF(df_with_weights, glueContext, "dynamic_frame")

# Saving the Glue DynamicFrame to S3
output_s3_path = "s3://my-data-source-bucket/your-output-folder/weightedratings.csv"

glueContext.write_dynamic_frame.from_options(
    dynamic_frame,  # Your DynamicFrame
    connection_type = "s3",
    connection_options = {
        "path": output_s3_path
    },
    format = "csv"  # Writing the data in CSV format
)

print(f"Data written to S3 path: {output_s3_path}")


Data written to S3 path: s3://my-data-source-bucket/your-output-folder/weightedratings.csv
